## Acoustic Integration - Influece of `emitter_terminate`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mitsuba as mi

from utils import shoebox_scene
from acoustic_params import EDC, T

plt.style.use('ggplot')

mi.set_variant('cuda_ad_acoustic')
mi.set_log_level(mi.LogLevel.Warn)

### Scene Construction

In [ ]:
def shoebox_scene_wrapper(**kwargs):
    scene = shoebox_scene(**kwargs)
    scene["integrator"]["emitter_terminate"] = kwargs["emitter_terminate"]
    return scene

### Simulation

In [ ]:
scattering = [0.0, 0.25, 0.5, 0.75, 1.0]
absorption = [0.1, 0.25, 0.5, 0.75, 0.9]

config = {
    "box_dim":     [25., 12., 7.],
    "mic_pos":     [ 9.,  6., 1.],
    "speaker_pos": [20.,  7., 2.],
    "speaker_radius": 0.1,
    "interference": False,

    "absorption": [(i + 1, a) for i, a in enumerate(absorption)],
    "scattering": 0.0, # default for now

    "wav_bins": len(absorption), # x
    "time_bins": 4000,           # y
    "max_time": 4,

    "integrator": "acousticpath",
    "max_depth": 150,
    "spp": 2**22, # 1024 * 4096 = 2^10 * ~time_bins
}

In [ ]:
df = pd.DataFrame()

for s in scattering:
    config['scattering'] = s

    config["emitter_terminate"] = False
    scene_dict = shoebox_scene_wrapper(**config)
    scene = mi.load_dict(scene_dict)
    hist_f = np.array(mi.render(scene))[:, :, 0]

    config["emitter_terminate"] = True
    scene_dict = shoebox_scene_wrapper(**config)
    scene = mi.load_dict(scene_dict)
    hist_t = np.array(mi.render(scene))[:, :, 0]

    for i, a in config["absorption"]:
        ar, sr = round(a + 0.01, 1), round(s + 0.01, 1)
        df.insert(0, f'a{ar:.1f}-s{sr:.1f}-f', hist_f[:, i-1])
        df.insert(0, f'a{ar:.1f}-s{sr:.1f}-t', hist_t[:, i-1])

    print(f"s = {s} - done")

df.insert(0, "time", np.linspace(0., config['max_time'], config['time_bins'], endpoint=False))
df.set_index("time", inplace=True, drop=True)

print(df.shape)

### Histogram and EDC Comparison

In [ ]:
for i, s in enumerate(scattering):
    fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(12, 3), layout="constrained")
    for j, a in enumerate(absorption):
        ar, sr = round(a + 0.01, 1), round(s + 0.01, 1)
        ax1.plot(df.index, df[f"a{ar}-s{sr}-f"] - df[f"a{ar}-s{sr}-t"], label=f"a={a}")
        with np.errstate(all="ignore"):
            ax2.plot(df.index, EDC(df[f"a{ar}-s{sr}-f"] / config["spp"]) - EDC(df[f"a{ar}-s{sr}-t"] / config["spp"]))
    ax1.legend()
    ax1.set_ylabel("Hist [a.U.]")
    ax2.set_ylabel("Energy [dB]")
    fig.supxlabel("time [s]")
    fig.suptitle(f"s = {s}")
    plt.show()

### T30 Comparison

In [ ]:
for i, s in enumerate(scattering):
    fig, axs = plt.subplots(1, 5, figsize=(20, 5), sharey=True, layout="constrained")
    for j, a in enumerate(absorption):
        ar, sr = round(a + 0.01, 1), round(s + 0.01, 1)
        etf = EDC(df[f"a{ar}-s{sr}-f"])
        ett = EDC(df[f"a{ar}-s{sr}-t"])

        etf_t30 = T(df.index, etf)
        ett_t30 = T(df.index, ett)

        axs[j].plot(df.index, etf, 'C0', label="false")
        axs[j].plot(df.index, ett, 'C1', label="true")

        axs[j].axvline(etf_t30, linestyle='dashed', color='C0', label='T30')
        axs[j].axvline(ett_t30, linestyle='dashed', color='C1')

        axs[j].axhline(-60., linestyle='dotted', color='C3')

        axs[j].set_xlim(-0.1, np.max([df.index[np.argmax(etf < -100.)], etf_t30, ett_t30]) * 1.05)
        axs[j].set_ylim(-105., 5.)

        axs[j].set_xlabel("Time [s]")
        axs[j].set_ylabel("Energy [dB]")

        axs[j].set_title(f"s = {s} - a = {a}")

    plt.legend()
    plt.show()